## Installing Haystack

To start, let's install the latest release of Haystack with `pip`:

In [1]:
!pip install --upgrade --quiet pip
!pip install --quiet farm-haystack[colab,faiss]==1.17.2

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
chex 0.1.82 requires numpy>=1.25.0, but you have numpy 1.23.5 which is incompatible.
google-cloud-aiplatform 0.6.0a1 requires google-api-core[grpc]<2.0.0dev,>=1.22.2, but you have google-api-core 2.11.1 which is incompatible.
google-cloud-automl 1.0.1 requires google-api-core[grpc]<2.0.0dev,>=1.14.0, but you have google-api-core 2.11.1 which is incompatible.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.52 which is incompatible.
kfp 2.0.1 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.
pymc3 3.11.5 requires numpy<1.22.2,>=1.15.0, but you have numpy 1.23.5 which is incompatible.
pymc3 3.11.5 requires scipy<1.8.0,>=1.7.3, but you have scip

In [2]:
import logging

logging.basicConfig(format='%(levelname)s - %(name)s -  %(message)s', level=logging.WARNING)
logging.getLogger('haystack').setLevel(logging.INFO)

## Initializing the DocumentStore

We'll start creating our question answering system by initializing a DocumentStore. A DocumentStore stores the Documents that the question answering system uses to find answers to your questions. In this tutorial, we're using the `FAISSDocumentStore`.

Let's initialize our DocumentStore.

In [3]:
from haystack.document_stores import FAISSDocumentStore

document_store = FAISSDocumentStore(embedding_dim=128, faiss_index_factory_str='Flat')

> To learn more about the DocumentStore and the different types of external databases that we support, see [DocumentStore](https://docs.haystack.deepset.ai/docs/document_store).

In [4]:
!apt-get install --quiet poppler-utils -y
print('pip install poppler utils complete.')

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libpoppler97 poppler-data
Suggested packages:
  ghostscript fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanese-gothic | fonts-ipafont-gothic fonts-arphic-ukai
  fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  libpoppler97 poppler-data poppler-utils
0 upgraded, 3 newly installed, 0 to remove and 40 not upgraded.
Need to get 2562 kB of archives.
After this operation, 16.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 poppler-data all 0.4.9-2 [1475 kB]
Ign:2 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libpoppler97 amd64 0.86.1-0ubuntu1.3
Ign:3 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 poppler-utils amd64 0.86.1-0ubuntu1.3
Err:2 http://security.ubuntu.com/ubuntu focal-updates/main amd64 libpoppler97 amd64 0.86.1-0ubuntu1.3
  404  Not

In [5]:
!sudo apt-get update
!sudo apt-get install --quiet poppler-utils -y
print('pip install poppler complete for real and for true')

Get:1 https://packages.cloud.google.com/apt cloud-sdk InRelease [6361 B]
Get:2 http://packages.cloud.google.com/apt gcsfuse-focal InRelease [1225 B]
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:5 https://packages.cloud.google.com/apt cloud-sdk/main amd64 Packages [616 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-updates/multiverse amd64 Packages [32.4 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1485 kB]
Get:10 http://security.ubuntu.com/ubuntu focal-security/multiverse amd64 Packages [29.7 kB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [3431 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3906 kB]
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/r

In [6]:
# Initialize a PDFToTextConverter to convert the PDF to text
from haystack.utils import convert_files_to_docs, fetch_archive_from_http, clean_wiki_text
from haystack.utils import fetch_archive_from_http, convert_files_to_docs
from haystack.nodes import PDFToTextConverter, PreProcessor, DensePassageRetriever
from haystack.nodes import TextConverter, PDFToTextConverter, DocxToTextConverter, PreProcessor

In [7]:
#replace this paths with your input Data path
pdf_paths = ['/kaggle/input/book-for-qa/book.pdf']

# Convert the PDF documents to Haystack-compatible format
pdf_converter = PDFToTextConverter(remove_numeric_tables=True, valid_languages=['en'])
pdf_docs = []

for pdf_path in pdf_paths:
    doc = pdf_converter.convert(file_path=pdf_path, meta=None)[0]
    pdf_docs.append(doc)

pdftotext version 0.86.1
Copyright 2005-2020 The Poppler Developers - http://poppler.freedesktop.org
Copyright 1996-2011 Glyph & Cog, LLC


In [8]:
# Initializing the PreProcessor to clean and split the text
preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by='word',
    split_length=100,
    split_respect_sentence_boundary=True,
)
processed_docs = preprocessor.process(pdf_docs)

Preprocessing:   0%|          | 0/1 [00:00<?, ?docs/s]

## Initializing the Retriever

We use a `DensePassageRetriever` and we invoke `update_embeddings` to index the embeddings of documents in the `FAISSDocumentStore`.

In [9]:
# Initialize a DensePassageRetriever
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model='vblagoje/dpr-question_encoder-single-lfqa-wiki',
    passage_embedding_model='vblagoje/dpr-ctx_encoder-single-lfqa-wiki',
)

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [10]:
document_store.write_documents(processed_docs)
document_store.update_embeddings(retriever)

Writing Documents:   0%|          | 0/11 [00:00<?, ?it/s]

Updating Embedding:   0%|          | 0/11 [00:00<?, ? docs/s]

Create embeddings:   0%|          | 0/16 [00:00<?, ? Docs/s]

## Initializing the Generator

we now initalize our Generator.

Here we use a `Seq2SeqGenerator` with the [*vblagoje/bart_lfqa*](https://huggingface.co/vblagoje/bart_lfqa) model.

In [11]:
from haystack.nodes import Seq2SeqGenerator

generator = Seq2SeqGenerator(model_name_or_path='vblagoje/bart_lfqa')
print('built generator.')

built generator.


## Initializing the Reader
we now initalize our Reader.

Here we use a FARMReader

In [12]:
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path='deepset/deberta-v3-large-squad2')
print('built the reader.')

built the reader.


## Initializing the Pipeline

With a Haystack `Pipeline` you can stick together your building blocks to a search pipeline.
Under the hood, `Pipelines` are Directed Acyclic Graphs (DAGs) that you can easily customize for your own use cases.
To speed things up, Haystack also comes with a few predefined Pipelines. One of them is the `GenerativeQAPipeline` that combines a Retriever and a Generator to answer our questions.
You can learn more about `Pipelines` in the [docs](https://docs.haystack.deepset.ai/docs/pipelines).

In [13]:
from haystack.pipelines import GenerativeQAPipeline
from haystack.pipelines import ExtractiveQAPipeline

generative_pipeline = GenerativeQAPipeline(generator, retriever)
extractive_pipeline = ExtractiveQAPipeline(reader, retriever)

## Ask Questions

In [14]:
query = 'Where did the bulls live?'
result = extractive_pipeline.run(query=query, params={'Retriever': {'top_k': 5}, 'Reader': {'top_k': 1}})
print(result)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

{'query': 'Where did the bulls live?', 'no_ans_gap': -7.116243243217468, 'answers': [<Answer {'answer': 'Where did the bulls live?\n___________________________________________________________________________\n____________________________________________________________________\nb. How many bulls were united', 'type': 'extractive', 'score': 0.023241665214300156, 'context': 'Where did the bulls live?\n___________________________________________________________________________\n____________________________________________________________________\nb. How many bulls were united', 'offsets_in_document': [{'start': 175, 'end': 375}], 'offsets_in_context': [{'start': 0, 'end': 200}], 'document_ids': ['296fde43f0dea180cde7938f42c9181d'], 'meta': {'_split_id': 2, 'vector_id': '0'}}>], 'documents': [<Document: {'content': 'If he fought with one bull, the other three would also join in. They were\nvery united. Then lion made a plan. He went to each one separately and told them that the other\nthre

In [15]:
list(result.keys())

['query',
 'no_ans_gap',
 'answers',
 'documents',
 'root_node',
 'params',
 'node_id']

In [16]:
print(result['answers'][0])

<Answer: answer='Where did the bulls live?
___________________________________________________________________________
____________________________________________________________________
b. How many bulls were united', score=0.023241665214300156, context='Where did the bulls live?
________________________...'>


In [17]:
piperun=generative_pipeline.run(
    query='Where did the bulls live?', params={'Retriever': {'top_k': 5}} #output varies with different top_k values
)
for answer in piperun['answers']:
    print(answer.answer)


I'm not sure if this is what you're looking for, but I'll give it a shot. The bulls were domesticated by the Romans. They were bred for their meat, not for their ability to fight.


In [18]:
piperun=generative_pipeline.run(
    query='Where did the bulls live?', params={'Retriever': {'top_k': 5}}
)
print(len(piperun['answers']))


1


In [19]:

piperun=generative_pipeline.run(
    query='What are Pointers in C++?', params={'Retriever': {'top_k': 5}}
)
piperun['answers'][0].answer

'Pointers in C++ are a special type of variable called a pointer. A pointer is a variable that can be used to describe a function. For example, if you have a function that returns a value, you can use a pointer to tell the function to return a value that is equal to the value of the pointer.'